# Weight Change Test Notebook

In [1]:
import pickle
from src.make_dataset import RandomForestExtended
from tqdm.notebook import tqdm
from scipy.stats import pearsonr

%run ../src/post_analysis.py

**0217**
- Test the `PredictAndEvalIndividual` class
- Test the weight change calculator functions

- To do:
    - Add more weigth smoothing methods.
    - Add more weight change calculator methods.
    - Tuning window size.

**0218**
- Accuracy statistics and WeightChange scatter plot on all use ids

**0303**
- Find ways to improve the estimation.

In [2]:
with open('../models/trained_models/urination-rf-sources-extended-embedding-0202.pkl', 'rb') as f:
    urinate_model = pickle.load(f)

with open('../models/trained_models/defecation-rf-sources-extended-embedding-0202.pkl', 'rb') as f:
    defecate_model = pickle.load(f)

In [3]:
from src.data import load_annotation

complete_urinate_ids = load_annotation.get_complete_ids(
    annotation_filename="C:/Users/Jiajun/Desktop/download-project/data/processed/Annotation.csv",
    category='Urination'
)

complete_defecate_ids = load_annotation.get_complete_ids(
    annotation_filename="C:/Users/Jiajun/Desktop/download-project/data/processed/Annotation.csv",
    category='Defecation'
)

complete_urinate_ids = complete_urinate_ids[(complete_urinate_ids > 1000) & (complete_urinate_ids < 2060)]

In [4]:
complete_defecate_ids = complete_defecate_ids[complete_defecate_ids > 1000]
complete_defecate_ids

array([1802, 1806, 1830, 1831, 1839, 1854, 1862, 1863, 1870, 1875, 1881,
       1882, 1890, 1893, 1898, 1904, 1912, 1915, 1919, 1923, 1926, 1930,
       1933, 1937, 1940, 1941, 1943, 1944, 1947, 1955, 1992, 1994, 1995,
       1999, 2001, 2002, 2010, 2013, 2014, 2021, 2022, 2024, 2028, 2030,
       2034, 2041, 2044, 2050, 2051, 2055, 2057, 2061, 2062, 2065, 2066,
       2068, 2069, 2072, 2073], dtype=int64)

## Urination

In [5]:
urinate_train_config = {
    'USE_IDS': [],
    'DATAFRAME_PATH': "C:/Users/Jiajun/Desktop/download-project/data/raw/data_frames",
    'ANNOTATION_PATH': "C:/Users/Jiajun/Desktop/download-project/data/processed/Annotation.csv",
    'FEATURE_NAMES': ['Max', 'Min', 'Mean', 'Median', 'LogVariance', 'LinearTrend'],
    'SOURCE_NAMES': ['TotalWeight', 'RadarSum', 'AudioDelay4'],
    'WINDOW_SECONDS': 2,
    'HOP_SECONDS': 1,
    'CATEGORY': 'Urination'
}

eval_config = {
    'USE_ID': 1875,
    'MODEL': urinate_model,
    'DATASET_TYPE': RandomForestExtended,
    'TRAIN_CONFIG': urinate_train_config,
    'THRESHOLD': 0.5,
    'CATEGORY': 'Urination',
    'DIFF_METHOD': 'EndMinusStart',
    'SMOOTH_METHOD': 'MedianFilter',
    'ANNOTATION_PATH': 'C:/Users/Jiajun/Desktop/download-project/data/processed/Annotation.csv'
}

In [6]:
res_urinate = []
res_urinate_updated_ids = []
for use_i in tqdm(complete_urinate_ids):
    try:
        eval_config_i = eval_config.copy()
        eval_config_i['USE_ID'] = use_i
        eval_i = PredictAndEvalIndividual(eval_config_i)
        annotated_intervals = eval_i.get_annotated_intervals()
        pred_intervals = eval_i.get_pred_intervals()
        annotated_weight_change = eval_i.get_annotated_weight_change()
        pred_weight_change = eval_i.get_pred_weight_change()
        total_length = eval_i.get_total_length_ts()
        
        res_urinate_updated_ids.append(use_i)
        res_urinate.append([total_length, annotated_intervals, pred_intervals, annotated_weight_change, pred_weight_change])
    except:
        print(f'Fail {use_i}')

  0%|          | 0/128 [00:00<?, ?it/s]

updating 1802
updating 1806
updating 1808
updating 1818
updating 1826
updating 1828
updating 1829
updating 1830
updating 1831
updating 1832
updating 1833
updating 1834
updating 1835
updating 1836
updating 1839
updating 1841
updating 1845
updating 1854
updating 1862
updating 1863
updating 1864
updating 1870
updating 1871
updating 1874
updating 1875
updating 1876
updating 1877
updating 1878
updating 1879
updating 1880
updating 1881
updating 1882
updating 1883
updating 1884
updating 1885
updating 1887
updating 1888
updating 1889
updating 1890
updating 1891
updating 1892
updating 1893
updating 1894
updating 1895
updating 1896
updating 1897
updating 1898
Fail 1898
updating 1904
updating 1906
updating 1911
updating 1912
updating 1913
updating 1914
updating 1915
updating 1916
updating 1917
updating 1918
updating 1919
updating 1920
updating 1921
updating 1922
updating 1923
updating 1924
updating 1925
updating 1926
updating 1927
updating 1928
updating 1929
updating 1930
updating 1931
updating 1

In [7]:
res_urinate_df = pd.DataFrame(
    res_urinate,
    columns=["Total Length", "Annotated Intervals", "Predicted Intervals",
             "Annotated Weight Change", "Predicted Weight Change"],
    index=res_urinate_updated_ids
)
res_urinate_df

,Total Length,Annotated Intervals,Predicted Intervals,Annotated Weight Change,Predicted Weight Change
1802,158,"[[47.0, 60.0]]","[[47.0, 56.0]]",0.349833,0.321833
1806,97,"[[46.0, 53.0]]",[],0.015333,0.000000
1808,88,"[[12.0, 34.0], [37.0, 39.0]]","[[12.0, 35.0]]",0.604500,0.604500
1818,55,"[[12.0, 25.0]]","[[11.96, 25.96]]",0.065833,0.032167
1826,65,"[[21.0, 32.0]]","[[20.96, 32.96]]",0.367000,0.367167
...,...,...,...,...,...
2053,50,"[[13.0, 26.0]]","[[12.919999999999998, 26.919999999999998]]",0.604333,0.621500
2054,47,"[[6.0, 19.0]]","[[6.96, 18.96]]",0.192500,0.192500
2055,234,"[[35.0, 39.0], [43.0, 53.0]]",[],0.149500,0.000000
2057,109,"[[27.0, 46.0], [51.0, 52.0]]","[[31.0, 43.0]]",0.091667,0.071667


In [8]:
res_urinate_df.to_pickle("./urinate_overall_evaluation_result_0304.pkl")

# Defecation Manual

In [ ]:
from src.post_analysis import EvalManual

In [ ]:
config = {
    'USE_ID': 1806,
    'FILTER_WINDOW_SIZE': 5,
    'THRESHOLD': 0.15,
    'EXTENSION_TIME': 0.8,
    'CATEGORY': 'Defecation',
    'DIFF_METHOD': 'EndMinusStart',
    'SMOOTH_METHOD': 'MedianFilter',
    'ANNOTATION_PATH': 'C:/Users/Jiajun/Desktop/download-project/data/processed/Annotation.csv'    
}

In [ ]:
res_defecate = []
res_defecate_updated_ids = []
for use_i in tqdm(complete_defecate_ids):
    try:
        config_i = config.copy()
        config_i['USE_ID'] = use_i
        eval_i = EvalManual(config_i)
        annotated_intervals = eval_i.get_annotated_intervals()
        pred_intervals = eval_i.get_pred_intervals()
        annotated_weight_change = eval_i.get_annotated_weight_change()
        pred_weight_change = eval_i.get_pred_weight_change()
        total_length = eval_i.get_total_length_ts()
        
        res_defecate_updated_ids.append(use_i)
        res_defecate.append([total_length, annotated_intervals, pred_intervals, annotated_weight_change, pred_weight_change])
    except:
        print(f'Fail {use_i}')

In [ ]:
res_defecate_df = pd.DataFrame(
    res_defecate,
    columns=["Total Length", "Annotated Intervals", "Predicted Intervals",
             "Annotated Weight Change", "Predicted Weight Change"],
    index=res_defecate_updated_ids
)
res_defecate_df

In [ ]:
res_defecate_df.to_pickle("./defecate_overall_evaluation_result_0218.pkl")

In [ ]:
res_defecate_df = res_defecate_df[res_defecate_df.index > 1000]